In [43]:
import pandas as pd
import requests
import re
from shapely.geometry import Point
import geopandas as gpd

In [44]:
def acquisition_bicimad (path_file_csv_bicimad):
    df_bicimad_raw= pd.read_csv(path_file_csv_bicimad)
    return df_bicimad_raw

path_file_csv_bicimad= "../data/bicimad_stations.csv"
df_bicimad_raw_test = acquisition_bicimad(path_file_csv_bicimad)
df_bicimad_raw_test

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry_type,geometry_coordinates
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,15,11,0,Point,"[-3.7018341, 40.4172137]"
1,2,Puerta del Sol B,1,1b,Puerta del Sol nº 1,1,0,30,22,4,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,Miguel Moya,2,2,Calle Miguel Moya nº 1,1,0,24,12,11,0,Point,"[-3.7058415, 40.4205886]"
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,11,0,Point,"[-3.7069171, 40.4302937]"
4,5,Malasaña,2,4,Calle Manuela Malasaña nº 5,1,0,24,12,12,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,266,Ciudad Universitaria 1,0,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,0,24,0,Point,"[-3.72699, 40.44375]"
260,267,Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,12,0,Point,"[-3.72693, 40.44342]"
261,268,Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,9,14,0,Point,"[-3.7272945, 40.4483322]"
262,269,Facultad Derecho,0,261,Avenida Complutense nº 23,1,0,24,5,18,0,Point,"[-3.72937, 40.45109]"


In [45]:
def lat (x):
    b= x.split(',')
    c= b[0].replace('[','')
    d= float(c)
    return d
    
def long (x):
    b= x.split(',')
    c= b[1].replace(']','')
    d= float(c)
    return d

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [46]:
df_bicimad_raw['Latitude.bicimad'] = df_bicimad_raw.apply(lambda x: long(x['geometry_coordinates']),axis=1)

In [47]:
def wrangling_bicimad (df_bicimad_raw):
    # I create the latitude and longitude columns
    df_bicimad_raw['Latitude.bicimad'] = df_bicimad_raw.apply(lambda x: long(x['geometry_coordinates']),axis=1)
    df_bicimad_raw['Longitude.bicimad'] = df_bicimad_raw.apply(lambda x: lat(x['geometry_coordinates']),axis=1)
    
    # I create the new column location applying the function to_mercator on latitude and longitude     
    df_bicimad_raw['location_bicimad']= df_bicimad_raw.apply(lambda x: to_mercator (x['Latitude.bicimad'], x['Longitude.bicimad']), axis=1)
    
    # I keep only the columns I am interested in
    df_bicimad= df_bicimad_raw[['name', 'address', 'Latitude.bicimad', 'Longitude.bicimad', 'location_bicimad']]
    return df_bicimad
    

In [48]:
wrangling_bicimad (df_bicimad_raw_test)

,name,address,Latitude.bicimad,Longitude.bicimad,location_bicimad
0,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,POINT (4499234.670122574 -412347.498528782)
2,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,POINT (4499599.340516399 -412820.3206106317)
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,POINT (4500679.707306497 -412940.3068148262)
4,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...
259,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,POINT (4502177.655770458 -415179.5220676512)
260,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,POINT (4502140.920338497 -415172.8287427756)
261,Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,POINT (4502687.743941171 -415213.4906984274)
262,Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,POINT (4502994.74083288 -415445.024322674)


In [49]:
# Centros deportivos 

In [50]:
def acquisition_sport_centers (url_madrid_sport_centers):
    r= requests.get(url_madrid_sport_centers).json()
    df_sport_centers_raw = pd.json_normalize(r['@graph'])
    return df_sport_centers_raw

In [51]:
url_madrid_sport_centers= 'https://datos.madrid.es/egob/catalogo/212808-0-espacio-deporte.json'
df_sport_centers_raw = acquisition_sport_centers (url_madrid_sport_centers)
#df_sport_centers_raw

#df_centros_deportivos= df_centros_deportivos.dropna() #elimina una row que tenía un valor nulo
#df_centros_deportivos

In [52]:
def wrangling_sport_centers (df_sport_centers_raw):
    # We first remove empty values 
    df_sport_centers_raw= df_sport_centers_raw.dropna()
    
    # I create the new column location_centros applying the function to_mercator on latitude and longitude 
    df_sport_centers_raw['location_centros']= df_sport_centers_raw.apply(lambda x: to_mercator (x['location.latitude'], x['location.longitude']), axis=1)
    
    # I keep only the columns I am interested in
    df_sport_centers= df_sport_centers_raw [['title', 'address.street-address', 'location.latitude', 'location.longitude', 'location_centros'] ]
    return df_sport_centers


In [53]:
wrangling_sport_centers (df_sport_centers_raw)

/Users/Izaskun/miniconda3/envs/bicimad/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,address.street-address,location.latitude,location.longitude,location_centros
0,Campo de Golf del Centro Nacional de Golf,CALLE ARROYO DEL MONTE 5,40.485508,-3.735279,POINT (4506826.154319164 -416104.2065551275)
1,Campo de Golf del Club de Golf Olivar de la Hi...,CALLE VIA DE DUBLIN 12,40.465351,-3.611293,POINT (4504582.255355831 -402273.7208142197)
2,Campos de Fútbol Ernesto Cotorruelo,CALLE VIA LUSITANA 5,40.381231,-3.724179,POINT (4495218.036163405 -414865.966552915)
3,Centro Comercial Dreams - Palacio Municipal de...,CALLE SILVANO 77,40.462913,-3.635734,POINT (4504310.842371287 -404999.9123872458)
4,Centro de Alto Rendimiento del Consejo Superio...,CALLE MARTIN FIERRO 5,40.436812,-3.731957,POINT (4501405.320516344 -415733.6645299961)
...,...,...,...,...,...
605,Piscina Real Canoe Madrid,CALLE PEZ VOLADOR 30,40.413487,-3.664876,POINT (4498808.844717023 -408250.5871617015)
606,Piscina SAGE Fitness,CALLE ESCALONA 59,40.397182,-3.761105,POINT (4496993.758844213 -418985.2758262118)
608,Pista de petanca Centro de Mayores La Vaguada,AVENIDA MONFORTE DE LEMOS 36,40.479033,-3.708264,POINT (4506105.338268181 -413090.5785481936)
609,Polideportivo Antonio Magariños,CALLE SERRANO 129,40.442522,-3.686096,POINT (4502040.979548661 -410617.7057179091)


In [54]:
def final_wrangling(df_bicimad_raw,df_sport_centers_raw):
    
    # Function to clean and separate values of latitude and longitude of bicimad
    def lat (x):
        b= x.split(',')
        c= b[0].replace('[','')
        d= float(c)
        return d
    def long (x):
        b= x.split(',')
        c= b[1].replace(']','')
        d= float(c)
        return d
    
    # Function to transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    def to_mercator(lat, long):
        c = gpd.GeoSeries([Point(lat, long)], crs=4326)
        c = c.to_crs(3857)
        return c
    
    # Function to return the distance in metres between two pseudo-mercator coordinates

    def distance_meters(location_centros, location_bicimad):
        return location_centros.distance(location_bicimad)
    
    #BICIMAD
    # I create the latitude and longitude columns 
    df_bicimad_raw['Latitude.bicimad'] = df_bicimad_raw.apply(lambda x: long(x['geometry_coordinates']),axis=1)
    df_bicimad_raw['Longitude.bicimad'] = df_bicimad_raw.apply(lambda x: lat(x['geometry_coordinates']),axis=1)
    
    # I create the new column location applying the function to_mercator on latitude and longitude     
    df_bicimad_raw['location_bicimad']= df_bicimad_raw.apply(lambda x: to_mercator (x['Latitude.bicimad'], x['Longitude.bicimad']), axis=1)
    
    # I keep only the columns I am interested in
    df_bicimad= df_bicimad_raw[['name', 'address', 'Latitude.bicimad', 'Longitude.bicimad', 'location_bicimad']]
    
    
    # SPORT CENTERS 
    # We first remove empty values 
    df_sport_centers_raw= df_sport_centers_raw.dropna()
    
    # I create the new column location_centros applying the function to_mercator on latitude and longitude 
    df_sport_centers_raw['location_centros']= df_sport_centers_raw.apply(lambda x: to_mercator (x['location.latitude'], x['location.longitude']), axis=1)
    
    # I keep only the columns I am interested in
    df_sport_centers= df_sport_centers_raw [['title', 'address.street-address', 'location.latitude', 'location.longitude', 'location_centros'] ]
    
    #MERGE BOTH DATAFRAMES
    
    df_merged = df_sport_centers.assign(key=0).merge(df_bicimad.assign(key=0),how='left', on = 'key')
    
    # Calculate the distance between the combination of all bicimad points and all sport center points  
    df_merged['Distance']= df_merged.apply(lambda x: distance_meters (x ['location_bicimad'], x ['location_centros']),axis=1 )
    
    # I create a table with the minimum distances grouped bye sport center
    min_distances= df_merged.groupby('title')['Distance'].min().reset_index()
    
    # I merge with the df_merged table to obtain more than the title and Distance columns
    df_final = min_distances.merge(df_merged, how='left', on = ['Distance', 'title'])
    return df_final


    
    

In [115]:
df_izas= final_wrangling (df_bicimad_raw_test, df_sport_centers_raw)

/Users/Izaskun/miniconda3/envs/bicimad/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [128]:
df_izas

,title,Distance,address.street-address,location.latitude,location.longitude,location_centros,key,name,address,Latitude.bicimad,Longitude.bicimad,location_bicimad
0,Campo de Golf del Centro Nacional de Golf,3887.705045,CALLE ARROYO DEL MONTE 5,40.485508,-3.735279,POINT (4506826.154319164 -416104.2065551275),0,Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,POINT (4502994.74083288 -415445.024322674)
1,Campo de Golf del Club de Golf Olivar de la Hi...,4843.269670,CALLE VIA DE DUBLIN 12,40.465351,-3.611293,POINT (4504582.255355831 -402273.7208142197),0,Carlota O’Neill,Calle Carlota O'Neill nº 32,40.443705,-3.648958,POINT (4502172.693570442 -406475.0626355441)
2,Campos de Fútbol Ernesto Cotorruelo,2074.787928,CALLE VIA LUSITANA 5,40.381231,-3.724179,POINT (4495218.036163405 -414865.966552915),0,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,40.398247,-3.716591,POINT (4497112.284980891 -414019.4641230708)
3,Centro Comercial Dreams - Palacio Municipal de...,2597.642883,CALLE SILVANO 77,40.462913,-3.635734,POINT (4504310.842371287 -404999.9123872458),0,Carlota O’Neill,Calle Carlota O'Neill nº 32,40.443705,-3.648958,POINT (4502172.693570442 -406475.0626355441)
4,Centro Deportivo Municipal María Jesús Rosa (...,2162.785163,AVENIDA MONFORTE DE LEMOS 38,40.478768,-3.708699,POINT (4506075.795342792 -413139.1196950942),0,Remonta,Calle Azucenas nº 1,40.463028,-3.697333,POINT (4504323.639518123 -411871.2121165185)
...,...,...,...,...,...,...,...,...,...,...,...,...
594,Piscina Real Canoe Madrid,503.950489,CALLE PEZ VOLADOR 30,40.413487,-3.664876,POINT (4498808.844717023 -408250.5871617015),0,Plaza de los Astros,Avenida de Nazaret nº 7,40.411448,-3.668909,POINT (4498581.757919108 -408700.4734607235)
595,Piscina SAGE Fitness,3869.554385,CALLE ESCALONA 59,40.397182,-3.761105,POINT (4496993.758844213 -418985.2758262118),0,Caramuel,Calle Caramuel nº 51,40.409136,-3.728534,POINT (4498324.476616229 -415351.7319152079)
596,Pista de petanca Centro de Mayores La Vaguada,2159.005542,AVENIDA MONFORTE DE LEMOS 36,40.479033,-3.708264,POINT (4506105.338268181 -413090.5785481936),0,Remonta,Calle Azucenas nº 1,40.463028,-3.697333,POINT (4504323.639518123 -411871.2121165185)
597,Polideportivo Antonio Magariños,162.774690,CALLE SERRANO 129,40.442522,-3.686096,POINT (4502040.979548661 -410617.7057179091),0,Serrano 113,Calle Serrano nº 113B,40.441060,-3.686090,POINT (4501878.206340224 -410617.0111451639)


In [127]:
df_izas_short= df_izas[['title','address.street-address', 'name', 'address']]
df_izas_short

df_izas_renamed= df_izas_short.rename({'title': 'Centro deportivo', 'address.street-address': 'Dirección del centro', 'name': 'Estación BiciMAD', 'Dirección BiciMAD': 'Station location'}, axis=1)
df_izas_renamed

,Centro deportivo,Dirección del centro,Estación BiciMAD,address
0,Campo de Golf del Centro Nacional de Golf,CALLE ARROYO DEL MONTE 5,Facultad Derecho,Avenida Complutense nº 23
1,Campo de Golf del Club de Golf Olivar de la Hi...,CALLE VIA DE DUBLIN 12,Carlota O’Neill,Calle Carlota O'Neill nº 32
2,Campos de Fútbol Ernesto Cotorruelo,CALLE VIA LUSITANA 5,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
3,Centro Comercial Dreams - Palacio Municipal de...,CALLE SILVANO 77,Carlota O’Neill,Calle Carlota O'Neill nº 32
4,Centro Deportivo Municipal María Jesús Rosa (...,AVENIDA MONFORTE DE LEMOS 38,Remonta,Calle Azucenas nº 1
...,...,...,...,...
594,Piscina Real Canoe Madrid,CALLE PEZ VOLADOR 30,Plaza de los Astros,Avenida de Nazaret nº 7
595,Piscina SAGE Fitness,CALLE ESCALONA 59,Caramuel,Calle Caramuel nº 51
596,Pista de petanca Centro de Mayores La Vaguada,AVENIDA MONFORTE DE LEMOS 36,Remonta,Calle Azucenas nº 1
597,Polideportivo Antonio Magariños,CALLE SERRANO 129,Serrano 113,Calle Serrano nº 113B


In [126]:
df2_izas= df_izas.rename
t= df_izas[df_izas['title'].isin(cat_values)][['title', 'address.street-address' ,'Distance', 'name', 'address']]

t2= t.rename({'title': 'Place of interest', 'address.street-address': 'Place address', 'name': 'BiciMAD station', 'address': 'Station location'}, axis=1)
t2

,Place of interest,Place address,Distance,BiciMAD station,Station location
577,Instalación Deportiva Municipal Básica de Riba...,PLAZAS INTERIORES ENTRE PLAZA RIBADEO 1 Y PLAZ...,2205.930306,Remonta,Calle Azucenas nº 1
598,WiZink Center,"CALLE JORGE JUAN 99 Avenida de Felipe II, s/n",223.496235,O'Donnell,Calle O'Donnell nº 50


In [110]:
t= df_izas[df_izas['title'].isin(cat_values)][['title', 'address.street-address' ,'Distance', 'name', 'address']]
t2= t.rename({'title': 'Place of interest', 'address.street-address': 'Place address', 'name': 'BiciMAD station', 'address': 'Station location'}, axis=1)

In [114]:
t2.insert(1, 'Types of place', 'Centro deportivo')
t2

,Place of interest,Types of place,Type of place,Place address,Distance,BiciMAD station,Station location
577,Instalación Deportiva Municipal Básica de Riba...,Centro deportivo,Centro deportivo,PLAZAS INTERIORES ENTRE PLAZA RIBADEO 1 Y PLAZ...,2205.930306,Remonta,Calle Azucenas nº 1
598,WiZink Center,Centro deportivo,Centro deportivo,"CALLE JORGE JUAN 99 Avenida de Felipe II, s/n",223.496235,O'Donnell,Calle O'Donnell nº 50


In [103]:
centers = input('Escribe el centro o los centros deportivos que te intersan:   ')

Escribe el centro o los centros deportivos que te intersan:   WiZink Center


In [104]:

centers_list= [centers]
print (centers_list)

['WiZink Center']


In [105]:
li= df_izas[df_izas['title'].isin(centers_list)][['title', 'address.street-address' ,'Distance', 'name', 'address']]

In [131]:
li= df_izas[df_izas['title'].isin(centers_list)]
li

,title,Distance,address.street-address,location.latitude,location.longitude,location_centros,key,name,address,Latitude.bicimad,Longitude.bicimad,location_bicimad
598,WiZink Center,223.496235,"CALLE JORGE JUAN 99 Avenida de Felipe II, s/n",40.423284,-3.672104,POINT (4499899.340417478 -409056.8845014666),0,O'Donnell,Calle O'Donnell nº 50,40.421315,-3.672497,POINT (4499680.180730613 -409100.6977278018)


In [132]:
df_sport_centers_raw['location_centros']= df_sport_centers_raw.apply(lambda x: to_mercator (x['location.latitude'], x['location.longitude']), axis=1)

KeyboardInterrupt: 